<center><h1>Шаблон оформления решения соревнования.</h1></center>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    f1_score, 
    accuracy_score,
    classification_report, 
)

# <center> 1. Подготовка обучающих данных </center>

## 1.1. Получение данных

In [ ]:
train_data = pd.read_csv("../../data/original/train.csv.csv", index_col=0)
assessment = train_data['assessment']

text = pd.read_csv("../../data/processed/text_df.csv", index_col=0)
tags = pd.read_csv("../../data/processed/tags_df.csv", index_col=0)
targets = pd.read_csv("../../data/processed/target_df.csv", index_col=0)

In [ ]:
text = text.fillna("")

## 1.2. Получение эмбеддингов текстов

In [ ]:
train_embeddings = ...

## 1.3. Формирование итоговых датасетов для обучения 

In [ ]:
final_train_data_1 = pd.concat([pd.DataFrame(assessment.values), pd.DataFrame(train_embeddings.values)], axis=1).values
final_train_data_2 = pd.concat([pd.DataFrame(assessment.values), pd.DataFrame(tags.values), pd.DataFrame(train_embeddings.values)], axis=1).values

# <center> 2. Обучение моделей </center>

# <center> 3. Подготовка тестовых данных </center> 

## 3.1. Получение данных

In [ ]:
test_data = pd.read_csv("../../data/original/test.csv.csv", index_col=0)
test_assessment = test_data['assessment']

test_data.head()

## 3.2. Предобработка текстовых данных

In [ ]:
import re
from bs4 import BeautifulSoup


def preprocessor(text):
    
    text = BeautifulSoup(text, "html.parser").get_text() # Удаляем HTML
    text = re.sub(r'http\S+', '', text)  # Удаление ссылок
    text = re.sub(r'\d+', '', text)  # Удаление номеров
    text = re.sub(r'\s+', ' ', text) # удаление лишних пробелов
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\.,?!\s]', '', text) # Удаление всех символов кроме значимых(буквы, пунктуация)
    text = text.lower()
    
    return text

In [ ]:
test_text = pd.DataFrame(test_data['text'].values, columns=['text'])
test_text = test_text.fillna("")
test_text['text'] = test_text['text'].apply(preprocessor)
test_text.head()

## 3.3. Получение эмбеддингов текстов

In [ ]:
test_embeddings = ...

## 3.4. Работа с признаком тегов

In [ ]:
test_data[test_data['tags'].isna()] = test_data[test_data['tags'].isna()].fillna('')

tags_set = {'ASSORTMENT',
 'CATALOG_NAVIGATION',
 'DELIVERY',
 'PAYMENT',
 'PRICE',
 'PRODUCTS_QUALITY',
 'PROMOTIONS',
 'SUPPORT'}

def string_to_set(string):
  elements = string.strip('{}').split(',')
  set_result = set(element.strip() for element in elements)
  return set_result


def make_tags_df(df, tags_set):
    tags_list = list(tags_set)
    tags_list = sorted(tags_list)
    new_df = pd.DataFrame(columns=tags_list)

    for _, row in df.iterrows():
        tags = string_to_set(row['tags'])
        for tag in tags_list:
            if tag in tags:
                new_df.loc[row.name, tag] = 1
            else:
                new_df.loc[row.name, tag] = 0
    return new_df


test_tags = pd.DataFrame(make_tags_df(test_data, tags_set).values, columns=sorted(list(tags_set)))
test_tags.head()

## 3.5. Итоговые тестовые данные и предсказания 

In [ ]:
final_test_data_1 = pd.concat([pd.DataFrame(test_assessment.values), pd.DataFrame(test_embeddings)], axis=1).values
final_test_data_2 = pd.concat([pd.DataFrame(test_assessment.values), pd.DataFrame(test_tags.values), pd.DataFrame(test_embeddings)], axis=1).values

In [ ]:
prediction_1 = model_1.predict(final_test_data_1)
prediction_2 = model_2.predict(final_test_data_2)

In [ ]:
def get_class_labels(array):
    
  class_labels = []
  for i in range(array.shape[0]):
    classes = np.where(array[i] == 1)[0]
    if len(classes) > 0:
      class_labels.append(" ".join(str(x) for x in classes))
    else:
      class_labels.append("")


  df = pd.DataFrame({"class_labels": class_labels})

  return df

answer_1 = get_class_labels(prediction_1)
answer_2 = get_class_labels(prediction_2)

In [ ]:
answer_1 = pd.DataFrame(answer_1.values, index=test_data['index'].values, columns=['target'])
answer_1.to_csv('../../results/submissions/n-1.csv')

In [ ]:
answer_2 = pd.DataFrame(answer_2.values, index=test_data['index'].values, columns=['target'])
answer_2.to_csv('../../results/submissions/n-2.csv')